In [2]:
!pip install requests
!pip install google-generativeai
!pip install chromadb
!pip install PyMuPDF

  Using cached chromadb-0.5.20-py3-none-any.whl.metadata (6.8 kB)
  Using cached onnxruntime-1.19.2-cp39-cp39-win_amd64.whl.metadata (4.7 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.28.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.49b2-py3-none-any.whl.metadata (2.1 kB)
  Using cached opentelemetry_sdk-1.28.2-py3-none-any.whl.metadata (1.5 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached opentelemetry_instrumentation_asgi-0.49b2-py3-none-any.whl.metadata (1.9 kB)
  Using cached opentelemetry_instrumentation-0.49b2-py3-none-any.whl.metadata (6.1 kB)
  Using cached opentelemetry_semantic_conventions-0.49b2-py3-none-any.whl.metadata (2.3 kB)
Using cached chromadb-0.5.20-py3-none-any.whl (617 kB)
Using cached onnxruntime-1.19.2-cp39-cp39-win_amd64.whl (11.1 MB)
Using cached opentelemetry_exporter_otlp_proto_grpc-1.28.2-py3-none-any.whl (18 kB)
Using cached opentelemetry_instrumentation_f

In [4]:
!pip install python-dotenv

In [3]:
import requests
import os
import PyPDF2
import fitz

In [2]:
from dotenv import load_dotenv
import os
import google.generativeai as genai

load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=api_key)

c:\Users\Dell\miniconda3\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
#checking for the gemini model used for text generation.
for model in genai.list_models():
  if "generateContent" in model.supported_generation_methods:
    print(model.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/learnlm-1.5-pro-experimental
models/gemini-exp-1114


In [13]:
def load_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text("text")  # Extracts continuous text
        text += "\n\n"  # Add paragraph spacing
    return text

In [14]:
text = load_pdf("../datas/word2vec.pdf")
print(text)

Efﬁcient Estimation of Word Representations in
Vector Space
Tomas Mikolov
Google Inc., Mountain View, CA
tmikolov@google.com
Kai Chen
Google Inc., Mountain View, CA
kaichen@google.com
Greg Corrado
Google Inc., Mountain View, CA
gcorrado@google.com
Jeffrey Dean
Google Inc., Mountain View, CA
jeff@google.com
Abstract
We propose two novel model architectures for computing continuous vector repre-
sentations of words from very large data sets. The quality of these representations
is measured in a word similarity task, and the results are compared to the previ-
ously best performing techniques based on different types of neural networks. We
observe large improvements in accuracy at much lower computational cost, i.e. it
takes less than a day to learn high quality word vectors from a 1.6 billion words
data set. Furthermore, we show that these vectors provide state-of-the-art perfor-
mance on our test set for measuring syntactic and semantic word similarities.
1
Introduction
Many current NLP 

In [15]:
def split_txt_to_chunk(text, max_len = 1500, chunk_overlap=200):
  chunks = []
  start = 0
  text_length = len(text)
  while start<text_length:
    end = start + max_len
    if end<text_length:
      end = text.rfind(" ", start, end) + 1

      if end <=start:
        end=start+max_len
    chunk = text[start:end].strip()

    if chunk:
      chunks.append(chunk)
    start = end - chunk_overlap
    if start >= text_length:
      break
  return chunks

In [16]:
chunks = split_txt_to_chunk(text, 1500, 200)
print(chunks)
print(len(chunks))

['Efﬁcient Estimation of Word Representations in\nVector Space\nTomas Mikolov\nGoogle Inc., Mountain View, CA\ntmikolov@google.com\nKai Chen\nGoogle Inc., Mountain View, CA\nkaichen@google.com\nGreg Corrado\nGoogle Inc., Mountain View, CA\ngcorrado@google.com\nJeffrey Dean\nGoogle Inc., Mountain View, CA\njeff@google.com\nAbstract\nWe propose two novel model architectures for computing continuous vector repre-\nsentations of words from very large data sets. The quality of these representations\nis measured in a word similarity task, and the results are compared to the previ-\nously best performing techniques based on different types of neural networks. We\nobserve large improvements in accuracy at much lower computational cost, i.e. it\ntakes less than a day to learn high quality word vectors from a 1.6 billion words\ndata set. Furthermore, we show that these vectors provide state-of-the-art perfor-\nmance on our test set for measuring syntactic and semantic word similarities.\n1\nIntr

In [3]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions

google_ef = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=api_key)
client = chromadb.PersistentClient(path = "embeddings/gemini")

collection = client.get_or_create_collection(name="pdf_rag", embedding_function=google_ef)

for i, d in enumerate(chunks):
  collection.add(documents=[d], ids=[str(i)])

NameError: name 'chunks' is not defined

In [18]:
def build_escaped_context(context):
    escaped_context = ""
    for item in context:
        if item.strip():  
            escaped_context += item.strip() + "\n\n"
    return escaped_context.strip() 

def find_relevant_context(query, db, n_results=3):
    results = db.query(query_texts=[query], n_results=n_results)
    
    if not results["documents"]:
        return "No relevant context found."
    
    relevant_documents = results["documents"]
    if len(relevant_documents) == 1:
        return build_escaped_context(relevant_documents[0])
    else:
        
        combined_context = "\n".join([doc.strip() for doc in relevant_documents[:n_results]])
        return build_escaped_context(combined_context.split("\n"))


In [19]:
results = find_relevant_context("what was the test set of this paper.", collection)

In [20]:
def create_prompt(query, context):
    prompt = f"""
    You are a knowledgeable assistant answering questions based strictly on the provided context.
    Do not make assumptions or include information not present in the context.
    If the context is insufficient to answer the question, respond with: "The context does not contain enough information to answer this question."
    
    Context:
    {context}
    
    Question:
    {query}
    
    Answer:
    """
    return prompt

In [21]:
def generate_answer(prompt):
  model = genai.GenerativeModel("gemini-1.5-pro-latest")
  result = model.generate_content(prompt)
  return result

In [22]:
prompt = create_prompt(query="What was the test set of this paper? ", context = results)
answer = generate_answer(prompt)

In [23]:
print(answer.text)

The context mentions a "new comprehensive test set" designed for measuring both syntactic and semantic regularities, but does not describe its contents.  The context also mentions that "the test set is available at", but the URL is incomplete.  Therefore, the context does not contain enough information to answer this question.

